# Testing the Pipeline from Class

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
import nltk

from pipeline import NLPPipe, tweet_clean1

from helper_functions import txt_to_df

import pickle
%load_ext autoreload
%autoreload 2

In [2]:
all_tweets = pd.read_pickle("all_tweets.pkl")

In [3]:
corpus_list = all_tweets['long_text'].tolist()
# Let's turn all of the tweets into a list, so our Pipeline
# can work with our data better

In [4]:
nlp = NLPPipe(vectorizer=CountVectorizer(), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer())
# Assign our Pipeline to a variable

In [6]:
nlp.fit(corpus_list)
nlp.transform(corpus_list);
# Fit the corpus and transform the corpus

In [7]:
pd.DataFrame(nlp.transform(corpus_list).toarray(), columns=nlp.vectorizer.get_feature_names()).head()
# It looks like we have a basic Document Term matrix, but all of the terms shown seem pretty wrong.
# A good first step would be to take out strings with number, but let's see if there is anything that seems off.

,00,000,00hsyobokk,00k9t0ock9,00pm,00uejqfvnl,01,0102,01nyvgqvhi,02,...,𝐰𝐨𝐫𝐭𝐡,𝕃𝕄𝔽𝔸𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆𝕆,𝗚𝗿𝗼𝘄𝘁𝗵𝗠𝗶𝗻𝗱𝘀𝗲𝘁,𝗮𝗰𝗰𝗼𝘂𝗻𝘁𝗮𝗯𝗶𝗹𝗶𝘁𝘆,𝗺𝗼𝗻𝗱𝗮𝘆𝘀,𝗻𝗼𝘃𝗲𝗺𝗯𝗲𝗿,𝘛𝘩𝘦𝘙𝘦𝘢𝘭𝘛𝘳𝘶𝘵𝘩𝘈𝘣𝘰𝘶𝘵𝘏𝘦𝘢𝘭𝘵𝘩,𝘤𝘰𝘮,𝙜𝙤𝙖𝙡𝙨,𝙧𝙪𝙡𝙚𝙨
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
nlp.vectorizer.vocabulary_['the']
# I didn't pass english stop words into the CountVectorizer, so that could be a good step to help out as well.

13747

In [5]:
nlp = NLPPipe(vectorizer=CountVectorizer(stop_words='english', max_df=0.80, min_df=10), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer(),
              cleaning_function=tweet_clean1)
# Now let's add those stopwords in there, and change the cleaning function.

In [6]:
nlp.fit(corpus_list)
nlp.transform(corpus_list);
# Fit the corpus and transform the corpus

In [35]:
dtm = pd.DataFrame(nlp.transform(corpus_list).toarray(), columns=nlp.vectorizer.get_feature_names())

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
dtm.sum(0).sort_values(ascending=False)
# I think taking out vegan, http and plantbas is okay, since all of these tweets are about these specific topics.

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


vegan       4411
http        3044
plantbas    1093
thi          752
amp          536
            ... 
brown         10
limit         10
later         10
late          10
nazi          10
Length: 1081, dtype: int64

In [7]:
stopwords = nltk.corpus.stopwords.words('english')

In [8]:
stopwords.append('vegan')
stopwords.append('vegan')
stopwords.append('http')

In [9]:
nlp = NLPPipe(vectorizer=CountVectorizer(stop_words=stopwords, max_df=0.80, min_df=10), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer(),
              cleaning_function=tweet_clean1)
# Now let's add those stopwords in there, and change the cleaning function.

In [10]:
nlp.fit(corpus_list)
dtm_tf = nlp.transform(corpus_list)

In [11]:
nlp2 = NLPPipe(vectorizer=TfidfVectorizer(stop_words=stopwords, max_df=0.80, min_df=10), 
              tokenizer=TreebankWordTokenizer().tokenize, 
              stemmer=PorterStemmer(),
              cleaning_function=tweet_clean1)
# Now let's add those stopwords in there, and change the cleaning function.

In [12]:
nlp2.fit(corpus_list)
nlp2.transform(corpus_list);
# Fit the corpus and transform the corpus

In [13]:
dtm_tfidf = nlp2.transform(corpus_list)

Let's do some basic topic modeling

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn

In [15]:
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf.fit(dtm_tf)

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=5, random_state=0)

In [67]:
tf_vectorizer = CountVectorizer(stop_words='english', max_df=0.80, min_df=10)

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, nlp.vectorizer)

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.157658 -0.096022       1        1  26.941325
4      0.098581  0.147453       2        1  20.297693
2     -0.187140  0.087787       3        1  19.024584
3     -0.230803 -0.100745       4        1  17.135270
0      0.161704 -0.038473       5        1  16.601129, topic_info=          Term         Freq        Total Category  logprob  loglift
749   plantbas  1071.000000  1071.000000  Default  30.0000  30.0000
45        anim   459.000000   459.000000  Default  29.0000  29.0000
687         nt   335.000000   335.000000  Default  28.0000  28.0000
503         im   207.000000   207.000000  Default  27.0000  27.0000
423         go   365.000000   365.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
613       make    47.438582   354.981290   Topic5  -5.0148  -0.2169
1121     world    37.725477   201.743488   Topic5  -5.2439   0.1190
43         amp    39.165427   523.561110   Topic5  -5.2064  -0.7972
1104       whi    34.881307   149.064966   Topic5  -5.3223   0.3433
1134      year    34.344462   153.268278   Topic5  -5.3378   0.3000

[362 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         5  0.929557     aabuzzeo
5         1  0.975704       access
6         5  0.941505       accord
8         5  0.929550  accupressur
24        1  0.966843          ago
...     ...       ...          ...
1134      1  0.508912         year
1134      2  0.182686         year
1134      3  0.006525         year
1134      4  0.078294         year
1134      5  0.221833         year

[609 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 1])

In [19]:
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(random_state=0)

In [20]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, nlp2.vectorizer)

/Users/joecowell/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.133778 -0.034935       1        1  14.058705
2     -0.154214 -0.010080       2        1  13.469103
4      0.089767  0.001194       3        1  11.119853
9      0.072544 -0.020694       4        1  10.616217
3     -0.263518 -0.077370       5        1  10.206932
5      0.050427 -0.020546       6        1   9.033844
0      0.055916 -0.011005       7        1   8.763866
7      0.004125  0.001030       8        1   8.634097
6      0.046303 -0.100358       9        1   8.190604
8     -0.035127  0.272763      10        1   5.906778, topic_info=               Term       Freq       Total Category  logprob  loglift
819           recip  54.000000   54.000000  Default  30.0000  30.0000
1070            via  26.000000   26.000000  Default  29.0000  29.0000
423              go  80.000000   80.000000  Default  28.0000  28.0000
45             anim  76.000000   76.000000  Default  27.0000  27.0000
794         pumpkin  20.000000   20.000000  Default  26.0000  26.0000
...             ...        ...         ...      ...      ...      ...
1122  worldveganday   6.710625   35.644397  Topic10  -4.6691   1.1592
36             also   6.538187   34.769743  Topic10  -4.6951   1.1580
572             let   5.610698   24.764229  Topic10  -4.8481   1.3443
462          health   5.348604   41.785762  Topic10  -4.8960   0.7733
749        plantbas   5.386228  146.387184  Topic10  -4.8889  -0.4733

[607 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         7  0.771728     aabuzzeo
2         3  0.184959      absolut
2         4  0.092479      absolut
2         7  0.092479      absolut
2        10  0.554876      absolut
...     ...       ...          ...
1137     10  0.550172       youtub
1138     10  0.869061  yrfatfriend
1140      5  0.809156        yummi
1140      8  0.101144        yummi
1141     10  0.879345    zbleumoon

[1574 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 10, 4, 6, 1, 8, 7, 9])